<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Знакомство-с-данными" data-toc-modified-id="Знакомство-с-данными-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Знакомство с данными</a></span></li><li><span><a href="#Предобработка" data-toc-modified-id="Предобработка-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Предобработка</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
!pip install -U scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: scikit-learn in /home/jovyan/.local/lib/python3.7/site-packages (1.0.1)


In [2]:
# Импорт библиотек
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.dummy import DummyClassifier
import torch
import transformers
from tqdm import tqdm
from tqdm import notebook
import json
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords as nltk_stopwords
from pymystem3 import Mystem
import re
from sklearn import decomposition
from sklearn.manifold import TSNE

import warnings
warnings.filterwarnings("ignore")

tqdm.pandas()

### Знакомство с данными

In [3]:
# Познакомимся с данными
df = pd.read_csv('/datasets/toxic_comments.csv')
display(df.head())
df.shape

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


(159571, 2)

In [4]:
# Проверим на пропуски и дубли
print(df.isnull().sum())
print(df.duplicated().sum())

text     0
toxic    0
dtype: int64
0


In [5]:
# Посмотрим на балансы классов
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

**Выводы:** Исходный датасет содержит 2 столбца: целевой признак toxic и тексты комментариев на английском языке. Значение целевого признака 0 соотвесвует нетоксичному кооментарию, а 1 - токсичному. Выборка содержит 159571 и является несбалансированной: позицивный комментариев больше

### Предобработка

In [6]:
# Напишем функции для лемматизации и очистки текста
def lemmatize(text):
    m = Mystem()
    lemm_list = m.lemmatize(' '.join(text))
    lemm_text = "".join(lemm_list)
        
    return lemm_text

def clear_text(text):    
    return ' '.join(re.sub(r'[^a-zA-Z`]', ' ', text).split())

def filter_words(text,stopwords): 
    return ' '.join([w for w in text.split() if not w.lower() in stopwords])

In [7]:
# Продем подготовку текста
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

df_init = df.copy()
df['text'] = df['text'].progress_apply(clear_text)
df['text'] = df['text'].progress_apply(filter_words,stopwords=stopwords )
# lemm_text = df['text'].progress_apply(lemmatize)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 159571/159571 [00:02<00:00, 63155.07it/s]


In [8]:
# Токенизируем тексты и созадим тренировочные и тестовые выборки
features_train, features_test, target_train, target_test = train_test_split(
        df['text'],df['toxic'], test_size=0.2, random_state=12345)

tfidf = TfidfVectorizer(stop_words=stopwords, max_df=0.7)
tfidf.fit(features_train)
features_train = tfidf.transform(features_train)
features_test = tfidf.transform(features_test)
print(f"Размер словаря {len(tfidf.vocabulary_)}")

Размер словаря 148283


## Обучение

In [9]:
# Обучим логистическую регрессию
LR = LogisticRegression(class_weight='balanced')
LR.fit(features_train, target_train)

LogisticRegression(class_weight='balanced')

In [10]:
# Подберем параметры для дерева решений
DTC = DecisionTreeClassifier(random_state=12345,class_weight='balanced')
params = {'max_depth':list(range(10,51,10))}
DTC = GridSearchCV(DTC, params, scoring='f1', cv=3, verbose=48, n_jobs=-1)
DTC.fit(features_train, target_train)
DTC.best_params_

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV 1/3; 1/5] START max_depth=10................................................
[CV 1/3; 1/5] END .................max_depth=10;, score=0.557 total time=  10.4s
[CV 2/3; 1/5] START max_depth=10................................................
[CV 2/3; 1/5] END .................max_depth=10;, score=0.543 total time=  10.3s
[CV 3/3; 1/5] START max_depth=10................................................
[CV 3/3; 1/5] END .................max_depth=10;, score=0.566 total time=  10.2s
[CV 1/3; 2/5] START max_depth=20................................................
[CV 1/3; 2/5] END .................max_depth=20;, score=0.602 total time=  17.9s
[CV 2/3; 2/5] START max_depth=20................................................
[CV 2/3; 2/5] END .................max_depth=20;, score=0.598 total time=  16.3s
[CV 3/3; 2/5] START max_depth=20................................................
[CV 3/3; 2/5] END .................max_depth=20;,

{'max_depth': 40}

In [11]:
# Подберем параметры для случайного леса
RFC = RandomForestClassifier(random_state=12345,class_weight='balanced')
params = {'max_depth':list(range(10,51,20)), 
         'n_estimators': list(range(80,121,20))}
RFC = GridSearchCV(RFC, params, scoring='f1', cv=3, verbose=72, n_jobs=-1)
RFC.fit(features_train, target_train)
RFC.best_params_

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV 1/3; 1/9] START max_depth=10, n_estimators=80...............................
[CV 1/3; 1/9] END max_depth=10, n_estimators=80;, score=0.358 total time=  12.0s
[CV 2/3; 1/9] START max_depth=10, n_estimators=80...............................
[CV 2/3; 1/9] END max_depth=10, n_estimators=80;, score=0.372 total time=  12.1s
[CV 3/3; 1/9] START max_depth=10, n_estimators=80...............................
[CV 3/3; 1/9] END max_depth=10, n_estimators=80;, score=0.367 total time=  12.2s
[CV 1/3; 2/9] START max_depth=10, n_estimators=100..............................
[CV 1/3; 2/9] END max_depth=10, n_estimators=100;, score=0.360 total time=  15.2s
[CV 2/3; 2/9] START max_depth=10, n_estimators=100..............................
[CV 2/3; 2/9] END max_depth=10, n_estimators=100;, score=0.375 total time=  15.0s
[CV 3/3; 2/9] START max_depth=10, n_estimators=100..............................
[CV 3/3; 2/9] END max_depth=10, n_estimators=10

{'max_depth': 50, 'n_estimators': 120}

In [12]:
# Подберем параметры для градиентного бустинга
LGBM = LGBMClassifier(random_state=12345,class_weight='balanced')
params = {'max_depth':list(range(10,31,10)), 
         'n_estimators': list(range(100,121,20))}
LGBM = GridSearchCV(LGBM, params, scoring='f1', cv=2, verbose=48, n_jobs=-1)
LGBM.fit(features_train, target_train)
LGBM.best_params_

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV 1/2; 1/6] START max_depth=10, n_estimators=100..............................
[CV 1/2; 1/6] END max_depth=10, n_estimators=100;, score=0.718 total time= 1.2min
[CV 2/2; 1/6] START max_depth=10, n_estimators=100..............................
[CV 2/2; 1/6] END max_depth=10, n_estimators=100;, score=0.719 total time= 1.1min
[CV 1/2; 2/6] START max_depth=10, n_estimators=120..............................
[CV 1/2; 2/6] END max_depth=10, n_estimators=120;, score=0.727 total time= 2.9min
[CV 2/2; 2/6] START max_depth=10, n_estimators=120..............................
[CV 2/2; 2/6] END max_depth=10, n_estimators=120;, score=0.726 total time= 1.5min
[CV 1/2; 3/6] START max_depth=20, n_estimators=100..............................
[CV 1/2; 3/6] END max_depth=20, n_estimators=100;, score=0.728 total time= 5.5min
[CV 2/2; 3/6] START max_depth=20, n_estimators=100..............................
[CV 2/2; 3/6] END max_depth=20, n_estimators

{'max_depth': 20, 'n_estimators': 120}

In [13]:
# Обучим модель-болванку
DC = DummyClassifier(strategy='stratified')
DC.fit(features_train, target_train)

DummyClassifier(strategy='stratified')

In [14]:
# Создадим списки, содержащие значения метрики
models = [LR, DTC, RFC, LGBM, DC]
train_f1 = []
test_f1 = []
# Обучим модели и оценим качество предсказаний
for model in models:    
    pred_test = model.predict(features_test)
    pred_train = model.predict(features_train)
    train_f1.append(f1_score(target_train, pred_train))
    test_f1.append(f1_score(target_test, pred_test))
    
# Создадим датафрейм с результатами
results = pd.DataFrame(
    [train_f1,test_f1], 
    columns=['Линейная регрессия', "Дерево решений", "Случайный лес", "Градиентный бустинг", 'Dummy модель'], 
    index=['F1 на тренировочной выборке','F1 на тестовой выборке' ])
display(results.head())

,Линейная регрессия,Дерево решений,Случайный лес,Градиентный бустинг,Dummy модель
F1 на тренировочной выборке,0.841834,0.759848,0.568262,0.776135,0.104965
F1 на тестовой выборке,0.755283,0.627735,0.476271,0.730095,0.098904


## Выводы

В рамках данного проекта была разработана модель, предсказывающая токсичность комментариев. Исходный датасет был несбалансированным, токсичных комментариев было меньше.

По итогу проекта были получены следующие результаты:
- Тексты комментариев были обработынны и очишены от лишних символов и частых слов
- Было обучено 4 модели и подобраны оптимальные гиперпараметры. Все 4 модели превзошли модель-болванку, но только 1 смогла побить требуемое значение метрики
- Лучшим результатом обладала логистическая регрессия со значением F1=0,76, затем идет градиентный бустинг со значением 0,73 и дерево решений с 0,63, замыкает список случайный лес с 0,48

Исходя из полученнх результатов, модель логистической регрессии предсказывает значением целевого признака с требуемой точностью и ее можно использовать для определения токсичных комментариев.